# Louvain
The _Louvain_ method of community detection is an algorithm for detecting communities in networks that relies upon a heuristic for maximizing the modularity.
Communities are groups of nodes within a network that are more densely connected to one another than to other nodes. 
A typical heuristic would be modularity, which quantifies the quality of an assignment of nodes to communities by evaluating how much more densely connected the nodes within a community are compared to how connected they would be in a random network.

The method consists of repeated application of two steps. 
The first step is a "greedy" assignment of nodes to communities, favoring local optimizations of modularity. 
The second step is the definition of a new coarse-grained network based on the communities found in the first step.
These two steps are repeated until no further modularity-increasing reassignments of communities are possible.

The _Louvain_ method achieves modularities comparable to pre-existing algorithms, typically in less time, so it enables the study of much larger networks.
It also reveals a hierarchy of communities at different scales, and this hierarchical perspective can be useful for understanding the global functioning of a network.
While there are pitfalls to interpreting the community structure uncovered by the Louvain Method, these difficulties are shared by all modularity optimization algorithms.[2]

First we'll import the Neo4j driver and Pandas libraries:


In [1]:
from neo4j.v1 import GraphDatabase, basic_auth
import pandas as pd
import os

Next let's create an instance of the Neo4j driver which we'll use to execute our queries.


In [2]:
host = os.environ.get("NEO4J_HOST", "bolt://localhost") 
user = os.environ.get("NEO4J_USER", "neo4j")
password = os.environ.get("NEO4J_PASSWORD", "neo")
driver = GraphDatabase.driver(host, auth=basic_auth(user, password))

Now let's create a sample graph that we'll run the algorithm against.


In [3]:
create_graph_query = '''
CREATE (nAlice:User {id:'Alice'})
,(nBridget:User {id:'Bridget'})
,(nCharles:User {id:'Charles'})
,(nDoug:User {id:'Doug'})
,(nMark:User {id:'Mark'})
,(nMichael:User {id:'Michael'})
CREATE (nAlice)-[:FRIEND]->(nBridget)
,(nAlice)-[:FRIEND]->(nCharles)
,(nMark)-[:FRIEND]->(nDoug)
,(nBridget)-[:FRIEND]->(nMichael)
,(nCharles)-[:FRIEND]->(nMark)
,(nAlice)-[:FRIEND]->(nMichael)
,(nCharles)-[:FRIEND]->(nDoug);
'''

with driver.session() as session:
    result = session.write_transaction(lambda tx: tx.run(create_graph_query))
    print("Stats: " + str(result.consume().metadata.get("stats", {})))

Stats: {'labels-added': 6, 'relationships-created': 7, 'nodes-created': 6, 'properties-set': 6}


Finally we can run the algorithm by executing the following query:


In [4]:
streaming_query = """
CALL algo.louvain.stream('User', 'FRIEND', {})
YIELD nodeId, community

MATCH (user:User) WHERE id(user) = nodeId

RETURN user.id AS user, community
LIMIT 20;
"""

with driver.session() as session:
    result = session.read_transaction(lambda tx: tx.run(streaming_query))        
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())

df

,user,community
0,Michael,1
1,Alice,1
2,Bridget,1
3,Charles,3
4,Doug,3
5,Mark,3
